# Week 4: Document-Based Stores (MongoDB)
### Student ID: [#####]
### Subtasks Done: [#,#,..]

### Task 1: Create a simple MongoDB out of this relational model

This is  a toy DB about movies and actors who played roles in these movies. This DB is consisted of  

- A "Person" table who has a unique id, and a name fields.

- Another "Movie" table that has a unique id, a title, a country where it was made, and a year when it was released.

- There is (m-n) or "many-many" relationship between these two tables (i.e basically, many actors can act in many movies, and the movie include many actors)
- Therefore, we use the "Roles" table in which we can deduct which person has acted in which movie, and what role(s) they played.

<img src="RDBSchema.png" alt="3" border="0">

#### Connect to the MongoDB server, and create a mongoDB with the name 'moviedb'

In [1]:
##YOUR CODE HERER
from pymongo import MongoClient
from random import randint
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')
myclient = MongoClient("mongodb://mongo:27017/") #Mongo URI format
mydb = myclient["moviedb"]

myclient.list_database_names()

['admin', 'config', 'local']

#### Create Person/Actor collection

In [2]:
##YOUR CODE HERER
person_coll = mydb["person"]
mydb.list_collection_names()


[]

#### Insert the data into the Person Table

In [3]:
personList = [
  { "id": 1, "name": "Charlie Sheen" },
  { "id": 2, "name": "Michael Douglas"},
  { "id": 3, "name": "Martin Sheen"},
  { "id": 4, "name": "Morgan Freeman"}
]

###YOUR CODE HERE
first_customers = person_coll.insert_many(personList)
print(first_customers.inserted_ids)

[ObjectId('634e73153b8939e789864779'), ObjectId('634e73153b8939e78986477a'), ObjectId('634e73153b8939e78986477b'), ObjectId('634e73153b8939e78986477c')]


#### Creating rest of Collections ("Movies", "Roles")

In [4]:
restcols = ["Movies","Roles"]

for col in restcols:
    mydb[col]

mydb.list_collection_names()

['person']

#### Inserting data into the movie Collection

In [5]:
moviescoll = mydb["Movies"]

movieList = [
  { "id": 1, "title": "Wall Street", "country":"USA","year":1987},
  { "id": 2, "title": "The American President", "country":"USA","year":1995},
  { "id": 3, "title": "The Shawshank Redemption", "country":"USA","year":1994},
]

###YOUR CODE HERE
movies_added = moviescoll.insert_many(movieList)
print(movies_added.inserted_ids)

[ObjectId('634e73173b8939e78986477d'), ObjectId('634e73173b8939e78986477e'), ObjectId('634e73173b8939e78986477f')]


#### Inserting data into the roles Collection

In [6]:
rolesCol = mydb["Roles"]

roleList = [
  { "personId": 1, "movieId": 1, "role":["Bud Fox"]},
  { "personId": 2, "movieId": 1, "role":["Carl Fox"]},
  { "personId": 3, "movieId": 1, "role":["Gordon Gekko"]},
  { "personId": 2, "movieId": 2, "role":["A.J. MacInerney"]},
  { "personId": 3, "movieId": 2, "role":["President Andrew Shepherd"]},
  { "personId": 4, "movieId": 3, "role":["Ellis Boyd 'Red' Redding"]}
]

###YOUR CODE HERE
roels_added = rolesCol.insert_many(roleList)
print(roels_added.inserted_ids)

[ObjectId('634e73193b8939e789864780'), ObjectId('634e73193b8939e789864781'), ObjectId('634e73193b8939e789864782'), ObjectId('634e73193b8939e789864783'), ObjectId('634e73193b8939e789864784'), ObjectId('634e73193b8939e789864785')]


### <font color ='green'>Just for your info</font>:

#### Another Way of Modeling this M-N model in Mongo would be using the Forien Keys 


* Movies


```[

{
	"_id": 1,
	"title":"Wall Street",
	"country":"USA",
	"year":1987,
	"persons":[1,2]
},

{
	"_id": 2,
	"title":"The American President",
	"country":"USA",
	"year":1995,
	"persons":[2]
}]
```
* Actors

```
[{
    "_id": 1,
    "name": "Charlie Sheen",
    "movies":[
    {"role": "Bud Fox", "movie_id":1}
    ]
},

{
    "_id": 2,
    "name": "Micheal Douglas",
    "movies":[
    {"role": "Gordon Geko", "movie_id":1},
    {"role": "President Andrew Shepherd", "movie_id":2}
    ]
}

] ```


#### Get all actors in your Mongo DB

In [7]:
###YOUR CODE HERE
actors = mydb.person.find({}, projection={'_id' : False})
for actor in actors:
    print(actor)
    
roles = mydb.Roles.find({}, projection={'_id' : False})
for role in roles:
    print(role)
    
movies = mydb.Movies.find({}, projection={'_id' : False})
for m in movies:
    print(m)

{'id': 1, 'name': 'Charlie Sheen'}
{'id': 2, 'name': 'Michael Douglas'}
{'id': 3, 'name': 'Martin Sheen'}
{'id': 4, 'name': 'Morgan Freeman'}
{'personId': 1, 'movieId': 1, 'role': ['Bud Fox']}
{'personId': 2, 'movieId': 1, 'role': ['Carl Fox']}
{'personId': 3, 'movieId': 1, 'role': ['Gordon Gekko']}
{'personId': 2, 'movieId': 2, 'role': ['A.J. MacInerney']}
{'personId': 3, 'movieId': 2, 'role': ['President Andrew Shepherd']}
{'personId': 4, 'movieId': 3, 'role': ["Ellis Boyd 'Red' Redding"]}
{'id': 1, 'title': 'Wall Street', 'country': 'USA', 'year': 1987}
{'id': 2, 'title': 'The American President', 'country': 'USA', 'year': 1995}
{'id': 3, 'title': 'The Shawshank Redemption', 'country': 'USA', 'year': 1994}


#### Get actors with names start with 'C' letter

In [8]:
###YOUR CODE HERE
actors = mydb.person.find({"name" : { "$regex" : "^C" }})
for actor in actors:
    print(actor)

{'_id': ObjectId('634e73153b8939e789864779'), 'id': 1, 'name': 'Charlie Sheen'}


#### Get all Movies sorted from recent to old! (get only the title and year fields)

In [9]:
###YOUR CODE HERE
from pymongo import DESCENDING as pymongoDescending
persons_Sorted=mydb.person.find({}).sort("created_at", pymongoDescending)
for person in persons_Sorted:
    print(person)

{'_id': ObjectId('634e73153b8939e789864779'), 'id': 1, 'name': 'Charlie Sheen'}
{'_id': ObjectId('634e73153b8939e78986477a'), 'id': 2, 'name': 'Michael Douglas'}
{'_id': ObjectId('634e73153b8939e78986477b'), 'id': 3, 'name': 'Martin Sheen'}
{'_id': ObjectId('634e73153b8939e78986477c'), 'id': 4, 'name': 'Morgan Freeman'}


#### Get all Movies released in the 90s (after year (1990) and before 2000) ordered from old to recent.

In [10]:
###YOUR CODE HERE
movies90 = mydb.Movies.find( 
    {"$and": [ 
        { "year": { "$gt": 1990 } },
        { "year": { "$lt": 2000 } } 
    ] } )

for movie in movies90:
    print(movie)

{'_id': ObjectId('634e73173b8939e78986477e'), 'id': 2, 'title': 'The American President', 'country': 'USA', 'year': 1995}
{'_id': ObjectId('634e73173b8939e78986477f'), 'id': 3, 'title': 'The Shawshank Redemption', 'country': 'USA', 'year': 1994}


#### Get Movies and Actors from your "movies" DB
* Hint : use the <code>'$lookup'</code> operator.
* The Result should be something like the following:
<code>
Charlie Sheen : Wall Street
Michael Douglas : Wall Street
Martin Sheen : Wall Street
Michael Douglas : The American President
Martin Sheen : The American President
Morgan Freeman : The Shawshank Redemption
</code>

In [11]:
###YOUR CODE HERE

results_agregation = mydb.Roles.aggregate([
    {
        "$lookup":{
            "from": "person",      
            "localField": "personId",  
            "foreignField": "id",
            "as": "person_info",         
        }
    },
    {   "$unwind":"$person_info" },
    {
        "$lookup":{
            "from": "Movies",      
            "localField": "movieId",  
            "foreignField": "id",
            "as": "movie_info",         
        }
    },
    {   "$unwind":"$movie_info" },
]);

for r in results_agregation:
    print(f"{r['person_info']['name']} : {r['movie_info']['title']}")

Charlie Sheen : Wall Street
Michael Douglas : Wall Street
Martin Sheen : Wall Street
Michael Douglas : The American President
Martin Sheen : The American President
Morgan Freeman : The Shawshank Redemption


#### For each Actor, get count of "Movies" he acted in.

In [12]:
###YOUR CODE HERE
results_agregation2 = mydb.person.aggregate([
    {
        "$lookup":{
            "from": "Roles",      
            "localField": "id",  
            "foreignField": "personId",
            "as": "roles_info",         
        }
    },
    {   "$unwind":"$roles_info" },
    { 
        "$group": { 
            "_id": "$name",
            "count": { "$sum": 1 } 
        } 
     },
]);
for r in results_agregation2:
    print(r)

{'_id': 'Charlie Sheen', 'count': 1}
{'_id': 'Michael Douglas', 'count': 2}
{'_id': 'Morgan Freeman', 'count': 1}
{'_id': 'Martin Sheen', 'count': 2}


#### In your DB, list the movies that every Actor played

In [13]:
###YOUR CODE HERE
results_agregation3 = mydb.Roles.aggregate([
    {
        "$lookup":{
            "from": "person",      
            "localField": "personId",  
            "foreignField": "id",
            "as": "person_info",         
        }
    },
    {   "$unwind":"$person_info" },
    {
        "$lookup":{
            "from": "Movies",      
            "localField": "movieId",  
            "foreignField": "id",
            "as": "movie_info",         
        }
    },
    {   "$unwind":"$movie_info" },
    { 
        "$group": { 
            "_id": "$person_info.name",
            "movies_acted": { "$push": "$movie_info.title" } 
        },
     },
]);
for r in results_agregation3:
    print(r)

{'_id': 'Charlie Sheen', 'movies_acted': ['Wall Street']}
{'_id': 'Michael Douglas', 'movies_acted': ['Wall Street', 'The American President']}
{'_id': 'Morgan Freeman', 'movies_acted': ['The Shawshank Redemption']}
{'_id': 'Martin Sheen', 'movies_acted': ['Wall Street', 'The American President']}


#### Get the Persons/Actors who acted in "Wall Street" movie
- Hint use `$lookup` , `$match` operators in the aggregation piepeline

In [14]:
###YOUR CODE HERE
results_agregation3 = mydb.Roles.aggregate([
    {
        "$lookup":{
            "from": "person",      
            "localField": "personId",  
            "foreignField": "id",
            "as": "person_info",         
        }
    },
    {   "$unwind":"$person_info" },
    {
        "$lookup":{
            "from": "Movies",      
            "localField": "movieId",  
            "foreignField": "id",
            "as": "movie_info",         
        }
    },
    {   "$unwind":"$movie_info" },
    { "$match": { "movie_info.title": { "$eq": "Wall Street" } } },
    { 
        "$group": { 
            "_id": "$movie_info.title",
            "actors": { "$push": "$person_info.name" } 
        },
     },
]);
for r in results_agregation3:
    print(r)

{'_id': 'Wall Street', 'actors': ['Charlie Sheen', 'Michael Douglas', 'Martin Sheen']}


#### Get the Movies in which "Micheal Douglas" has played a role in

In [15]:
###YOUR CODE HERE
results_agregation4 = mydb.Roles.aggregate([
    {
        "$lookup":{
            "from": "person",      
            "localField": "personId",  
            "foreignField": "id",
            "as": "person_info",         
        }
    },
    {   "$unwind":"$person_info" },
    {
        "$lookup":{
            "from": "Movies",      
            "localField": "movieId",  
            "foreignField": "id",
            "as": "movie_info",         
        }
    },
    {   "$unwind":"$movie_info" },
    { "$match": { "person_info.name": { "$regex": "Michael Douglas" } } },
    { 
        "$group": { 
            "_id": "$person_info.name",
            "movies_acted": { "$push": "$movie_info.title" } 
        },
     },
]);
for r in results_agregation4:
    print(r)

{'_id': 'Michael Douglas', 'movies_acted': ['Wall Street', 'The American President']}


#### Get count of "Movies" in your DB

In [16]:
###YOUR CODE HERE
mydb.Movies.count_documents({})


3

#### update the year of the 'Wall Street' movie was released in to be 2000(which is not true BTW :)
- Show that movie before and After updating it

In [18]:
###YOUR CODE HERE
print("+ + before + +")
res_mov = mydb.Movies.find_one({"title":"Wall Street"})
pprint(res_mov)

print("+ + after + +")

mydb.Movies.update_one({"title":"Wall Street"},{"$set": {"year":2000}  })

res_mov = mydb.Movies.find_one({"title":"Wall Street"})
pprint(res_mov)


+ + before + +
{'_id': ObjectId('634e73173b8939e78986477d'),
 'country': 'USA',
 'id': 1,
 'title': 'Wall Street',
 'year': 1987}
+ + after + +
{'_id': ObjectId('634e73173b8939e78986477d'),
 'country': 'USA',
 'id': 1,
 'title': 'Wall Street',
 'year': 2000}


####  Delete all the persons with names start with 'M' letter.

In [19]:
###YOUR CODE HERE
print("\n //////////////////BEFORE//////")
for cust in mydb.person.find({}):
    print(cust)
mydb.person.delete_many({"name" : { "$regex" : "^M" }})

print("\n //////////////////AFTER//////")

for cust in mydb.person.find({}):
    print(cust)


 //////////////////BEFORE//////
{'_id': ObjectId('634e73153b8939e789864779'), 'id': 1, 'name': 'Charlie Sheen'}
{'_id': ObjectId('634e73153b8939e78986477a'), 'id': 2, 'name': 'Michael Douglas'}
{'_id': ObjectId('634e73153b8939e78986477b'), 'id': 3, 'name': 'Martin Sheen'}
{'_id': ObjectId('634e73153b8939e78986477c'), 'id': 4, 'name': 'Morgan Freeman'}

 //////////////////AFTER//////
{'_id': ObjectId('634e73153b8939e789864779'), 'id': 1, 'name': 'Charlie Sheen'}


### Task 2: Extend your Mongo-"MovieDB" 

Imagine now that we are going to extend our DB with new movies, actors, even with new directors.

- We add <b>**"The matrix"**</b> movie which was released in <b> USA, (1999)</b>, and has a new property/field "Tagline" <b>("Welcome to the Real World")</b>.
 
- We will also add 4 new actors (Person):
    - **"Keanu Reeves"** who was born in (1964). <font color='green'>Note:</font> "born" property is also new.
    - **"Carrie-Anne Moss"** who was born in (1967).
    - **"Laurence Fishburne"** who was born in (1960).
    - **"Hugo Weaving"** who was born in (1960).
    
- Moreover, we add 2 directors (Person) :
    - **"Lilly Wachowski"**, born in (1967)
    - **"Lana Wachowski"**, born in(1965)
- For these directors specify one more label/field as ("Director"). (You can add this while inserting the director documents)
    
 
- We will also create a new <b>collection "Directed" </b> that is directed from the later 2 directors to "the Matrix" movie.

#### Add the Movie "The Matrix" with the provided data to the Movies collection

In [21]:
###YOUR CODE HERE
movies_added = moviescoll.insert_one({ "id": 4, "title": "The Matrix",
                                       "country":"USA","year":1999,
                                       "tagline":"Welcome to the Real World"})
print(movies_added.inserted_id)

634ef7003b8939e789864787


#### Insert the new 4 actors to the person collection

In [22]:
#Notice, How is easy to add a new field compared to the RDB
newActorList = [
  { "id": 5, "name": "Keanu Reeves", "born":1964 },
  { "id": 6, "name": "Carrie-Anne Moss", "born":1967},
  { "id": 7, "name": "Laurence Fishburne", "born":1960},
  { "id": 8, "name": "Hugo Weaving", "born":1960}
]

###YOUR CODE HERE
newActors = person_coll.insert_many(newActorList)
print(newActors.inserted_ids)

[ObjectId('634ef7e33b8939e789864788'), ObjectId('634ef7e33b8939e789864789'), ObjectId('634ef7e33b8939e78986478a'), ObjectId('634ef7e33b8939e78986478b')]


#### Insert the new 2 directors to the person collection

In [24]:
###YOUR CODE HERE
directorsList = [
  { "id": 9, "name": "Lilly Wachowski", "born":1967, "label":"Director"},
  { "id": 10, "name": "Lana Wachowski", "born":1965, "label":"Director"},
]

###YOUR CODE HERE
newDirectors = person_coll.insert_many(directorsList)
print(newDirectors.inserted_ids)

[ObjectId('634ef8cd3b8939e78986478c'), ObjectId('634ef8cd3b8939e78986478d')]


#### Create the "Directed" collection, and insert the data into it 

In [25]:
###YOUR CODE HERE
directedCol = mydb["Directed"]

directedList = [
  { "personId": 9, "movieId": 4 },
  { "personId": 10, "movieId": 4 },
]

###YOUR CODE HERE
dir_added = directedCol.insert_many(directedList)
print(dir_added.inserted_ids)

[ObjectId('634ef93f3b8939e78986478e'), ObjectId('634ef93f3b8939e78986478f')]


#### Get only the directors from the person collection (i.e. persons marked with the label "Director")

In [28]:
###YOUR CODE HERE
dirs = mydb.person.find( { "label": { "$eq": "Director" } } )

for d in dirs:
    print(d)

{'_id': ObjectId('634ef8cd3b8939e78986478c'), 'id': 9, 'name': 'Lilly Wachowski', 'born': 1967, 'label': 'Director'}
{'_id': ObjectId('634ef8cd3b8939e78986478d'), 'id': 10, 'name': 'Lana Wachowski', 'born': 1965, 'label': 'Director'}


#### Perform a query that get persons (names, and born year) who Directed "The Matrix" movie.

In [37]:
###YOUR CODE HERE
results_agregation3 = mydb.Directed.aggregate([
    {
        "$lookup":{
            "from": "person",      
            "localField": "personId",  
            "foreignField": "id",
            "as": "person_info",         
        }
    },
    {   "$unwind":"$person_info" },
    {
        "$lookup":{
            "from": "Movies",      
            "localField": "movieId",  
            "foreignField": "id",
            "as": "movie_info",         
        }
    },
    {   "$unwind":"$movie_info" },
    { "$match": { "movie_info.title": { "$eq": "The Matrix" } } },
    { 
        "$group": { 
            "_id": "$movie_info.title",
            "director": { "$push": { "name" : "$person_info.name", "born" : "$person_info.born" } } 
        },
     },
]);
for r in results_agregation3:
    print(r)

{'_id': 'The Matrix', 'director': [{'name': 'Lilly Wachowski', 'born': 1967}, {'name': 'Lilly Wachowski', 'born': 1967}, {'name': 'Lana Wachowski', 'born': 1965}, {'name': 'Lana Wachowski', 'born': 1965}]}


 ## How long did it take you to solve the homework?
 
Please answer as precisely as you can. It does not affect your points or grade in any way. It is okey, if it took 0.5 hours or 24 hours. The collected information will be used to improve future homeworks.

<font color="red"><b>Answer:</b></font> 12 h

**<center> <font color='red'>THANK YOU FOR YOUR EFFORT!</font></center>**